In [1]:
import pyspark
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib import patches
import numpy as np
from math import sqrt

sc = pyspark.SparkContext.getOrCreate()

In [2]:
rdd = sc.textFile('subsetData.csv', use_unicode=False).cache()
list(enumerate(rdd.first().split(',')))

[(0, 'SessionID'),
 (1, 'SessionDate'),
 (2, 'SessionDuration'),
 (3, 'SessionFrequency'),
 (4, 'RecordID'),
 (5, 'Timestamp'),
 (6, 'timeIntervalSince1970'),
 (7, 'GyroX'),
 (8, 'GyroY'),
 (9, 'GyroZ'),
 (10, 'AccX'),
 (11, 'AccY'),
 (12, 'AccZ'),
 (13, 'MagX'),
 (14, 'MagY'),
 (15, 'MagZ')]

In [4]:
rdd = rdd.filter(lambda x: not x.startswith('SessionID')) #remove header
print (rdd.first())

0,38:58.8,0:23,60,0,38:58.8,1540931939,-0.171710773,-0.44572688,-0.103311209,0.071121216,-0.13835144,-1.048599243,40.74794006,28.28944397,-76.89674377


In [5]:
print rdd.take(10)

['0,38:58.8,0:23,60,0,38:58.8,1540931939,-0.171710773,-0.44572688,-0.103311209,0.071121216,-0.13835144,-1.048599243,40.74794006,28.28944397,-76.89674377', '0,38:58.8,0:23,60,0,38:58.9,1540931939,-0.135502702,-0.069837139,-0.062163506,0.024291992,-0.188186646,-1.009124756,40.6590271,28.46276855,-76.73902893', '0,38:58.8,0:23,60,0,38:58.9,1540931939,-0.058512579,0.157536365,-0.035130024,0.054092407,-0.153060913,-1.031112671,40.920578,28.33343506,-76.729599', '0,38:58.8,0:23,60,0,38:58.9,1540931939,0.010897655,0.135293643,-0.049095375,0.083969116,-0.136428833,-1.029907227,40.95025635,28.17259216,-76.5778656', '0,38:58.8,0:23,60,0,38:58.9,1540931939,0.02978213,0.011873171,-0.065315091,0.090942383,-0.140869141,-1.003616333,40.95025635,28.17259216,-76.5778656', '0,38:58.8,0:23,60,0,38:58.9,1540931939,-0.10691633,-0.048637844,-0.022522353,0.076782227,-0.139648438,-0.988525391,40.78399658,28.4384613,-76.57702637', '0,38:58.8,0:23,60,0,38:58.9,1540931939,-0.173546224,-0.073879817,-0.041812961,0

In [6]:
#same funtion & logic concepts as the pandas notebook
def square(x):
    return x**2

def result(x,y,z):
    return abs(sqrt(x + y + z))

def getPositionVector(partitionId, records):
    import csv
    reader = csv.reader(records)
    for row in reader:
        if row[9] != 's':
            (gyroX, gyroY, gyroZ) = (square(float(row[7])), square(float(row[8])), square(float(row[9])))
            (accX, accY, accZ) = (square(float(row[10])), square(float(row[11])), square(float(row[12])))
            (magX, magY, magZ) = (square(float(row[13])), square(float(row[14])), square(float(row[15])))
            yield (result(gyroX, gyroY, gyroZ), result(accX, accY, accZ), result(magX, magY, magZ))

positionVectors = rdd.mapPartitionsWithIndex(getPositionVector)
                    
                                     

In [7]:
#def sum_partitions(iterator):
#    parts = np.array(list(iterator))
#    yield np.sum(parts, 0)

#positionVectors.mapPartitions(sum_partitions).collect()


In [8]:
#get the sum of each column
#returns (sum of gyro vectors, sum of acc vectors, sum of mag vectors)
pathSum = positionVectors.reduce(lambda x,y: [x[0] + y[0], x[1] + y[1], x[2] + y[2]])
print pathSum

[6.626323437859389, 33.13902414280171, 3006.3054821527594]
